# imports

In [ ]:
import os
import boto3
import json

In [ ]:
def get_boto_client(service, **kwargs):
    return boto3.client(
        service,
        aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
        aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
        **kwargs
    )

client = get_boto_client('iam')

### validate we can make an API call

In [ ]:
role_name = 'my-ec2-role'
resp1 = client.get_role(RoleName=role_name)
resp1

### read resource

In [ ]:
resp2 = client.get_instance_profile(InstanceProfileName=role_name)
resp2

### convert to CFN expected format

In [ ]:
def instance_profile_to_cfn(d:dict):
    """
    Args:
        d = boto3.client.get_instance_profile response
    
    https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/aws-resource-iam-instanceprofile.html
    
    Type: AWS::IAM::InstanceProfile
    Properties:
      InstanceProfileName: String
      Path: String
      Roles: 
        - String
    """
    return {
        "Type": "AWS::IAM::InstanceProfile",
        "DeletionPolicy": "Retain",
        "Properties": {
          "InstanceProfileName": d['InstanceProfile']['InstanceProfileName'],
          "Path": d['InstanceProfile']['Path'],
          "Roles": [
            x['RoleName'] for x in d['InstanceProfile']['Roles']
          ]
        }
    }

instance_profile_to_cfn(resp2)

### create-change-set

In [ ]:
cfn_client = get_boto_client('cloudformation')

### initial template we are importing has drift

In [ ]:
def get_template(resources: dict) -> dict:
    return {
        "AWSTemplateFormatVersion": "2010-09-09",
        "Resources": resources
    }

template = get_template(
    resources={
            "MyInstanceProfile": instance_profile_to_cfn(resp2)
        })

template

In [ ]:
instance_profile_name = role_name
instance_profile_name

In [ ]:
resp = cfn_client.create_change_set(
    StackName='MyImportStack2',
    TemplateBody=json.dumps(template),
    ChangeSetType='IMPORT',
    ChangeSetName="MyImportStack002",
    Capabilities=['CAPABILITY_NAMED_IAM'],
    ResourcesToImport=[
   {
      "ResourceType":"AWS::IAM::InstanceProfile",
      "LogicalResourceId":"MyInstanceProfile",
      "ResourceIdentifier":{
         "InstanceProfileName": instance_profile_name
      }
   }]
)

resp

In [ ]:
resp2 = cfn_client.describe_change_set(
    StackName='MyImportStack2',
    ChangeSetName="MyImportStack002",
)

resp2

### cfn stack will be in REVIEW_IN_PROGRESS until we make this call

In [ ]:
resp3 = cfn_client.execute_change_set(
    StackName='MyImportStack2',
    ChangeSetName="MyImportStack002",
)

resp3

### we now see Drift status is DRIFTED but Role is unchanged in AWS, so that's ok

In [ ]:
resp4 = cfn_client.detect_stack_drift(
    StackName='MyImportStack2'
)
resp4

In [ ]:
resp5 = cfn_client.describe_stack_drift_detection_status(
    StackDriftDetectionId=resp4['StackDriftDetectionId']
)
resp5

In [ ]:
assert resp5['StackDriftStatus'] == 'IN_SYNC'